In [35]:
#Collect and import ten documents
#I tried to segment this off so it was not pulling such a long part of the destination, but I got errors every other way I sliced it.

import pandas as pd
import os
import numpy as np

documents = []
path = '/Users/smargar/design/exercises/text2'

filenames=sorted([os.path.join(path, fn) for fn in os.listdir(path)])
print(len(filenames)) # count files in corpus
print(filenames[:10]) # print names of 1st ten files in corpus

10
['/Users/smargar/design/exercises/text2/Aenied_The.txt', '/Users/smargar/design/exercises/text2/Full_text_of_The_Nibelungenlied.txt', '/Users/smargar/design/exercises/text2/Odyssey.txt', '/Users/smargar/design/exercises/text2/Provided_by _he_Internet_Classics_Archive.txt', '/Users/smargar/design/exercises/text2/The_Project_Gutenberg_eBook_of_The_Divine_Comedy.txt', '/Users/smargar/design/exercises/text2/The_Rime_of_the_Ancient_Mariner.txt', '/Users/smargar/design/exercises/text2/The_Waste_Land.txt', '/Users/smargar/design/exercises/text2/beowulf_translation_by_seamus_heaney.txt', '/Users/smargar/design/exercises/text2/paradiselost.txt', '/Users/smargar/design/exercises/text2/paradiseregaind.txt']


In [36]:

import sklearn.feature_extraction.text as text

In [37]:
vectorizer=text.CountVectorizer(input='filename', stop_words="english", min_df=1.0)


In [38]:
dtm=vectorizer.fit_transform(filenames).toarray() # defines document term matrix

vocab=np.array(vectorizer.get_feature_names())